<a href="https://colab.research.google.com/github/skybristol/GeoArchiveSummer2021/blob/main/Working_with_a_Trello_archive.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import json
import glob

In [9]:
trello_export_file = next((i for i in glob.glob("TrelloExport*")), None)
if trello_export_file:
    trello_export_data = json.load(open(trello_export_file, "r"))
trello_export_data

{'actions': [{'appCreator': None,
   'data': {'board': {'id': '60c0f996418c5946dd8fef27',
     'name': 'Summer Intern Tasks',
     'shortLink': 'Ul8kFazU'},
    'card': {'id': '60ddc2e361d72f1aac31ced8',
     'idShort': 7,
     'name': 'Nail down product plans from the project and write draft titles/abstracts',
     'shortLink': 'ZMY9RulF'},
    'checkItem': {'id': '60ddc53500dbd5518f2a1c75',
     'name': 'Draft titles/abstracts for the products',
     'state': 'incomplete'},
    'checklist': {'id': '60ddc4aa2bd6dd5235ae6e76',
     'name': 'Publishing to-dos'}},
   'date': '2021-07-15T17:32:34.783Z',
   'id': '60f07132eb113681722e8e9f',
   'idMemberCreator': '60c12117d15e4106d568a456',
   'limits': {},
   'memberCreator': {'activityBlocked': False,
    'avatarHash': None,
    'avatarUrl': None,
    'fullName': 'Jimmy Swift',
    'id': '60c12117d15e4106d568a456',
    'idMemberReferrer': '60c0f9436b13e0107c712c25',
    'initials': 'JS',
    'nonPublic': {},
    'nonPublicAvailable': True